In [1]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import itertools
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page=requests.get(url)
doc=lh.fromstring(page.content)
#inspected table elements to find that table data is stored between HTML code <tr>...</tr>
tr_elements=doc.xpath('//tr')
col=[]
i=0
print(len(tr_elements))
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print((i,name))
    col.append((name,[]))


185
(1, 'Postal Code\n')
(2, 'Borough\n')
(3, 'Neighbourhood\n')


In [2]:
#first row is header, data is stored on the second row onwards
#
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    if len(T)!=3:
        break
    i=0
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content()
        col[i][1].append(data)
        i+=1
dict={title:column for (title,column) in col}
#convert dictionary into a dataframe
tor_neighborhoods=pd.DataFrame(dict)
tor_neighborhoods.columns=tor_neighborhoods.columns.str.replace('\n','')
tor_neighborhoods['Postal Code']=tor_neighborhoods['Postal Code'].str.replace('\n','')
tor_neighborhoods['Borough']=tor_neighborhoods['Borough'].str.replace('\n','')
tor_neighborhoods['Neighbourhood']=tor_neighborhoods['Neighbourhood'].str.replace('\n','')
tor_neighborhoods.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
#drop cells with a borough that is 'Not assigned'
tor_neighborhoods=tor_neighborhoods[tor_neighborhoods.Borough != 'Not assigned']
#If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
tor_neighborhoods['Neighbourhood'] = np.where(tor_neighborhoods['Neighbourhood']=='Not assigned',tor_neighborhoods['Borough'],tor_neighborhoods['Neighbourhood'])
tor_neighborhoods=tor_neighborhoods[['Postal Code','Borough','Neighbourhood']]
tor_neighborhoods.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
tor_neighborhoods.shape

(104, 3)